In [2]:
import wandb
import pandas as pd
from collections import Counter

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: lage. Use `wandb login --relogin` to force relogin


True

The main 'geoname' table has the following fields :
---------------------------------------------------
    geonameid         : integer id of record in geonames database
    name              : name of geographical point (utf8) varchar(200)
    asciiname         : name of geographical point in plain ascii characters, varchar(200)
    alternatenames    : alternatenames, comma separated, ascii names automatically transliterated, convenience attribute from alternatename table, varchar(10000)
    latitude          : latitude in decimal degrees (wgs84)
    longitude         : longitude in decimal degrees (wgs84)
    feature class     : see http://www.geonames.org/export/codes.html, char(1)
    feature code      : see http://www.geonames.org/export/codes.html, varchar(10)
    country code      : ISO-3166 2-letter country code, 2 characters
    cc2               : alternate country codes, comma separated, ISO-3166 2-letter country code, 200 characters
    admin1 code       : fipscode (subject to change to iso code), see exceptions below, see file admin1Codes.txt for display names of this code; varchar(20)
    admin2 code       : code for the second administrative division, a county in the US, see file admin2Codes.txt; varchar(80)
    admin3 code       : code for third level administrative division, varchar(20)
    admin4 code       : code for fourth level administrative division, varchar(20)
    population        : bigint (8 byte int)
    elevation         : in meters, integer
    dem               : digital elevation model, srtm3 or gtopo30, average elevation of 3''x3'' (ca 90mx90m) or 30''x30'' (ca 900mx900m) area in meters, integer. srtm processed by cgiar/ciat.
    timezone          : the iana timezone id (see file timeZone.txt) varchar(40)
    modification date : date of last modification in yyyy-MM-dd format

In [3]:
wandb.init(
    project="woher",
    job_type="data-prep"
)

In [4]:
geoname_columns = ["geonameid", "name", "asciiname", "alternatenames", "latitude", "longitude", \
                   "feature_class", "feature_code", "country_code", "cc2", "admin1_code", "admin2_code", \
                   "admin3_code", "admin4_code", "population", "elevation", "dem", "timezone", "modification_date"]
df_raw = pd.read_csv("gen1/training/data/cities500.txt", sep='\t', header=None, names=geoname_columns, index_col=0)
df_raw

/var/folders/13/vxllz0kn71s1swfjzsnb92th0000gn/T/ipykernel_3061/3808433244.py:4: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv("gen1/training/data/cities500.txt", sep='\t', header=None, names=geoname_columns, index_col=0)


,name,asciiname,alternatenames,latitude,longitude,feature_class,feature_code,country_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,modification_date
geonameid,,,,,,,,,,,,,,,,,,
3038999,Soldeu,Soldeu,NaN,42.57688,1.66769,P,PPL,AD,NaN,02,NaN,NaN,NaN,602,NaN,1832,Europe/Andorra,2017-11-06
3039154,El Tarter,El Tarter,"Ehl Tarter,Эл Тартер",42.57952,1.65362,P,PPL,AD,NaN,02,NaN,NaN,NaN,1052,NaN,1721,Europe/Andorra,2012-11-03
3039163,Sant Julià de Lòria,Sant Julia de Loria,"San Julia,San Julià,Sant Julia de Loria,Sant J...",42.46372,1.49129,P,PPLA,AD,NaN,06,NaN,NaN,NaN,8022,NaN,921,Europe/Andorra,2013-11-23
3039604,Pas de la Casa,Pas de la Casa,"Pas de la Kasa,Пас де ла Каса",42.54277,1.73361,P,PPL,AD,NaN,03,NaN,NaN,NaN,2363,2050.0,2106,Europe/Andorra,2008-06-09
3039678,Ordino,Ordino,"Ordino,ao er di nuo,orudino jiao qu,Ордино,オルデ...",42.55623,1.53319,P,PPLA,AD,NaN,05,NaN,NaN,NaN,3066,NaN,1296,Europe/Andorra,2018-10-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895269,Beitbridge,Beitbridge,"Bajtbridz,Bajtbridzh,Beitbridge,Beitbridzas,Be...",-22.21667,30.00000,P,PPL,ZW,NaN,07,NaN,NaN,NaN,26459,NaN,461,Africa/Harare,2013-03-12
895308,Beatrice,Beatrice,Beatrice,-18.25283,30.84730,P,PPL,ZW,NaN,04,NaN,NaN,NaN,1647,NaN,1307,Africa/Harare,2018-05-09
895417,Banket,Banket,"Banket,Banket Junction",-17.38333,30.40000,P,PPL,ZW,NaN,05,NaN,NaN,NaN,9641,NaN,1277,Africa/Harare,2013-03-12


# Dataset Cleanup

In [5]:
def str_contains(string, test_chars):
    return any(char in test_chars for char in string)

# Remove names containing a character that occurs less than a minimum amount
def remove_least_common_chars(df, name_column, min_char_count=10):
    c = Counter(df[name_column].str.cat())
    total_char_counts = c.most_common()
    too_uncommon_chars = [c[0] for c in total_char_counts if c[1] < min_char_count]
    return df.drop(df.index[df[name_column].apply(lambda x: str_contains(x, too_uncommon_chars)).values]) # Remove remaining names that have a double quote in them


In [13]:
unwanted_columns = ["alternatenames", "feature_class", "feature_code", \
                    "cc2", "admin1_code", "admin2_code", "admin3_code", "admin4_code", \
                    "population", "elevation", "dem", "timezone", "modification_date"]
min_char_count = 100
name_column = "asciiname"

df = df_raw.drop(columns=unwanted_columns)

print("Cleaning up dataset, before starting we have:")
print(f'{len(df)} city names from {len(set(df["country_code"].values))} countries')

# Remove or replace bad stuff
df = df.dropna()
#df.loc[:, name_column] = df[name_column].str.lower() # Only lowercase for now
df.loc[:, name_column] = df[name_column].str.replace(r"\(.*\)","", regex=True) # Remove substrings within parenthesis
df.loc[:, name_column] = df[name_column].str.replace(r"\[.*\]","", regex=True) # Remove substrings within brackets
df.loc[:, name_column] = df[name_column].str.replace(r'\".*\"',"", regex=True) # Remove substrings within quotes
df.loc[:, name_column] = df[name_column].str.replace(r"\'.*\'","", regex=True) # Remove substrings within quotes
df = df.drop(df.index[df[name_column].str.contains('"').values]) # Remove remaining names that have a double quote in them
df[name_column] = df[name_column].map(lambda s: s.split("/")[0]) # If names has alt-name after slash, pick first
df[name_column] = df[name_column].str.strip() # Strip whitespace

df = remove_least_common_chars(df, name_column, min_char_count)


print(f'{len(df)} city names from {len(set(df["country_code"].values))} countries')
print("... is what we ended up with at the end.")

unique_chars = sorted(list(set(df[name_column].str.cat())))
print(f'{len(unique_chars)} unique characters')

df.head(5)


Cleaning up dataset, before starting we have:
192528 city names from 246 countries
192236 city names from 245 countries
... is what we ended up with at the end.
58 unique characters


,name,asciiname,latitude,longitude,country_code
geonameid,,,,,
3038999,Soldeu,Soldeu,42.57688,1.66769,AD
3039154,El Tarter,El Tarter,42.57952,1.65362,AD
3039163,Sant Julià de Lòria,Sant Julia de Loria,42.46372,1.49129,AD
3039604,Pas de la Casa,Pas de la Casa,42.54277,1.73361,AD
3039678,Ordino,Ordino,42.55623,1.53319,AD


In [14]:
artifact = wandb.Artifact(name='cleaned-cities', type='dataset')
artifact.add(wandb.Table(dataframe=df), name="clean")
wandb.run.log_artifact(artifact)

<Artifact cleaned-cities>

In [15]:
wandb.run.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job
